In [4]:
import json
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from oandapyV20.contrib.factories import InstrumentsCandlesFactory

client = oandapyV20.API(access_token='4f3028fb81c7d0251ce5c0dbdffe1c43-c9445425731b0f7b53e78228e12d9745')

instrument = "DE30_EUR"
params = {
    "from": "2019-01-01T00:00:00Z",
    "granularity": "H1",
    "count": 4999
}

def cnv(r, h):
    # get all candles from the response and write them as a record to the filehandle h
    for candle in r.get('candles'):
        ctime = candle.get('time')[0:19]
        try:
            rec = "{time},{complete},{o},{h},{l},{c},{v}".format(
                time=ctime,
                complete=candle['complete'],
                o=candle['mid']['o'],
                h=candle['mid']['h'],
                l=candle['mid']['l'],
                c=candle['mid']['c'],
                v=candle['volume'],
            )
        except Exception as e:
            print(e, r)
        else:
            h.write(rec+"\n")

datafile = "./{}.{}.out".format(instrument, params['granularity'])
with open(datafile, "w") as O:
    n = 0
    for r in InstrumentsCandlesFactory(instrument=instrument, params=params):
        rv = client.request(r)
        cnt = len(r.response.get('candles'))
        print("REQUEST: {} {} {}, received: {}".format(r, r.__class__.__name__, r.params, cnt))
        n += cnt
        cnv(r.response, O)
    print("Check the datafile: {} under /tmp!, it contains {} records".format(datafile, n))

REQUEST: v3/instruments/DE30_EUR/candles InstrumentsCandles {'granularity': 'H1', 'includeFirst': True, 'from': '2019-01-01T00:00:00Z', 'to': '2019-07-28T07:00:00Z'}, received: 2954
REQUEST: v3/instruments/DE30_EUR/candles InstrumentsCandles {'granularity': 'H1', 'includeFirst': True, 'from': '2019-07-28T07:00:00Z', 'to': '2019-08-22T17:41:29Z'}, received: 378
Check the datafile: ./DE30_EUR.H1.out under /tmp!, it contains 3332 records
